# 🧠 Reinforcement Learning with the NIM Game
Let's teach our AI how to win a simple game using Q-learning.

## 🎮 The NIM Game Rules
- Start with 21 sticks.
- Each player takes 1, 2, or 3 sticks on their turn.
- The player who takes the **last stick loses**.

We'll train an AI to get smarter over time!

In [ ]:
MAX_STICKS = 21
ACTIONS = [1, 2, 3, 4]

## 🧠 Step 1: Create a Q-table
We’ll use a dictionary to store the AI’s knowledge — the expected value (Q) of taking each action in every possible state.

In [ ]:
Q = {}

## 🎲 Step 2: Action Choice
Let’s write a function that chooses an action. We’ll use **epsilon-greedy** — random at first, smarter later.

In [ ]:

import random

def choose_action(state, epsilon):
    if state not in Q:
        Q[state] = {a: 0 for a in ACTIONS}
    if random.random() < epsilon:
        return random.choice([a for a in ACTIONS if a <= state])
    return max(Q[state], key=Q[state].get)


## 💡 Step 3: Q-Value Update Rule
We’ll update the Q-values using this formula:
```
Q(s,a) = Q(s,a) + alpha * (reward + gamma * max(Q(s') - Q(s,a))
```

In [ ]:
import random

def choose_action(state, epsilon):
    if state not in Q:
        Q[state] = {a: 0 for a in ACTIONS}
    if random.random() < epsilon:
        return random.choice([a for a in ACTIONS if a <= state])
    return max(Q[state], key=Q[state].get)

# Define the update_q function here
def update_q(state, action, reward, next_state, alpha, gamma):
    # Ensure the next state exists in the Q table
    if next_state not in Q:
        # Assuming 0 is a terminal state with no possible actions
        Q[next_state] = {a: 0 for a in ACTIONS if a <= next_state}
        # For a terminal state like 0 sticks, there are no future actions, so max_q_next is 0
        max_q_next = 0 if next_state == 0 else max(Q[next_state].values())
    else:
         # Get the maximum Q value for the next state
         # Handle the case where next_state might be a terminal state (0 sticks)
        max_q_next = 0 if next_state == 0 else max(Q[next_state].values())


    # Calculate the temporal difference target
    target = reward + gamma * max_q_next

    # Calculate the error (TD error)
    error = target - Q[state][action]

    # Update the Q-value
    Q[state][action] += alpha * error


def train(episodes=10000, epsilon=0.3, alpha=0.1, gamma=0.9):
    for _ in range(episodes):
        state = MAX_STICKS
        last_state, last_action = None, None

        while state > 0:
            action = choose_action(state, epsilon)
            next_state = state - action

            # Update Q-value for the previous state and action if available
            if last_state is not None:
                # The reward for the previous state's action is 0 as the game is ongoing
                update_q(last_state, last_action, 0, state, alpha, gamma)

            last_state = state
            last_action = action

            if next_state == 0:
                # Bot took the last stick => bot loses (rewarded +1, although typically -1 for losing)
                # Let's follow the original code's logic and use +1 here for the state where the bot wins
                update_q(state, action, 1, next_state, alpha, gamma)
                break

            # Opponent's turn
            valid_opponent_actions = [a for a in ACTIONS if a <= next_state]
            if not valid_opponent_actions:
                 # If no valid actions for opponent, it means the bot's last move left 0 or fewer sticks.
                 # This condition should ideally not be reached if next_state check is correct,
                 # but adding a safety update with reward 0.
                 update_q(last_state, last_action, 0, next_state, alpha, gamma)
                 break


            # The opponent takes a random valid action
            opponent_action = random.choice(valid_opponent_actions)
            state = next_state - opponent_action # The state after the opponent's move


            if state <= 0:
                # Bot wins (opponent took last stick).
                # The reward is -1 as per the comment, punishing the bot for the move
                # that led to this winning state for the bot.
                # We update the Q value for the bot's *last* move that led to the state *before* the opponent's winning move.
                update_q(last_state, last_action, -1, state, alpha, gamma)
                break

## 🔁 Step 4: Training Loop
Now we’ll play lots of games where the AI learns from experience.

## 🚀 Train the AI!

In [ ]:
train()

In [ ]:
print(Q)

{21: {1: 0.6617523364268455, 2: 0.6847312049552602, 3: 0.6933591470971776, 4: 0.7020998916935944}, 19: {1: 0.4956067897042622, 2: 0.714770838222653, 3: 0.6068703388666056, 4: 0.5551990716318721}, 16: {1: 0.7289984769464327, 2: 0.721712176461752, 3: 0.7387649994031291, 4: 0.7476556632093796}, 13: {1: 0.7769580667002252, 2: 0.7859687452305363, 3: 0.8099999999999987, 4: 0.709398057894402}, 9: {1: 0.7041066299154006, 2: 0.8062768887103409, 3: 0.7469470403075547, 4: 0.899999999999999}, 5: {1: 0.34049603415999274, 2: -0.15910211283148273, 3: -0.35877876390093866, 4: -0.9988209815422262}, 3: {1: -0.38905635979667463, 2: -0.9999999999895721, 3: 0.9999999999999996, 4: 0}, 0: {}, 18: {1: 0.7155751452264201, 2: 0.6788476781050494, 3: 0.6875553077042885, 4: 0.6616735552235886}, 15: {1: 0.723050127690517, 2: 0.746061161370678, 3: 0.7515792474493079, 4: 0.7793287323551874}, 11: {1: 0.8099999999999987, 2: 0.7587399076234712, 3: 0.6964877069515032, 4: 0.7774550640594291}, 6: {1: 0.899999999999999, 2: 

## 🧪 Let’s play against the AI!

In [ ]:

def play():
    state = MAX_STICKS
    while state > 0:
        print(f"Sticks left: {state}")
        move = int(input("Your move (1–3): "))
        state -= move
        if state <= 0:
            print("You took the last stick. You lose!")
            return
        if state in Q:
            ai_move = max(Q[state], key=Q[state].get)
        else:
            ai_move = random.choice([a for a in ACTIONS if a <= state])
        print(f"AI takes {ai_move} stick(s).")
        state -= ai_move
        if state <= 0:
            print("AI took the last stick. You win!")
            return


In [ ]:
play()

Sticks left: 21
Your move (1–3): 3
AI takes 1 stick(s).
Sticks left: 17
Your move (1–3): 3
AI takes 4 stick(s).
Sticks left: 10
Your move (1–3): 2
AI takes 3 stick(s).
Sticks left: 5
Your move (1–3): 2
AI takes 3 stick(s).
AI took the last stick. You win!


## 🎉 Summary
You just trained an agent to play a game using trial-and-error. That’s the magic of Reinforcement Learning!